In [1]:
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
!nvidia-smi

Thu Dec 03 22:31:12 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 442.50       Driver Version: 442.50       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Quadro P2000       WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   38C    P8    N/A /  N/A |     78MiB /  4096MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
import torch.nn as nn
import torch
import torch.optim as optim

class Net(nn.Module):
    def __init__(self, conv_kernel_size_list, rnn_hidden_size, nconv_size, kernel_mode):
        super(Net, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=6, out_channels=32, kernel_size=conv_kernel_size_list[0])
        self.conv2 = nn.Conv1d(in_channels=6, out_channels=32, kernel_size=conv_kernel_size_list[1])
        self.conv3 = nn.Conv1d(in_channels=6, out_channels=32, kernel_size=conv_kernel_size_list[2])
        self.batchnorm = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size=2)
        self.drop1 = nn.Dropout(p=0.5)
        self.gru = nn.GRU(input_size=32, hidden_size=rnn_hidden_size, num_layers=2, batch_first=True, dropout=0.5)
        self.drop2 = nn.Dropout(p=0.5)
        self.fc_nconv = nn.Linear(nconv_size, 2)
        self.fc = nn.Linear(rnn_hidden_size + 2, 1)
        self.kernel_mode = kernel_mode
    
    def forward(self, conv_x, nconv_x):
        """
        x: (batch, 6, seq)
        """
        
        if self.kernel_mode == "multi-kernel":
            h0 = torch.cat((self.conv1(conv_x), self.conv2(conv_x), self.conv3(conv_x)), -1) # (batch, 32, seq  - (kernel size - 1))
        else:
            h0 = self.conv1(conv_x)
        h0 = self.batchnorm(h0)
        h0 = self.relu(h0)
        h0 = self.pool(h0)
        # Input (batch, seq, feature)
        h0 = self.drop1(h0)
        o, _ = self.gru(h0.transpose(1, 2))
        # o (batch, seq  - (kernel size - 1), rnn_hidden_size)
        nconv = self.fc_nconv(nconv_x)
        score = self.fc(torch.cat((self.drop2(o[:, -1]), nconv), -1))
        # self.sigmoid(score)
        # (batch,)
        return score.squeeze(1)

In [3]:
def get_non_conv_feature(keystroke_segment):
    total_token_count, back_count, lshift_count, rshift_count = 0, 0, 0, 0
    capital_count, lctrl_count, rctrl_count, left_right_count = 0, 0, 0, 0
    negUD_count, negUU_count = 0, 0
    if len(keystroke_segment) > 1 and len(keystroke_segment[0]) == 8 and len(keystroke_segment[-1]) == 8:
        start_timestamp = float(keystroke_segment[0][6])
        end_timestamp = float(keystroke_segment[-1][7])
    else:
        start_timestamp = 0.0
        end_timestamp = 0.0

    feature = []
    for keystrokes in keystroke_segment[2:]:
        fst_key_name, sec_key_name, fst_du, sec_du, sec_fst_du, sec_fst_dd, fst_timstamp, sec_timestamp = keystrokes[0], keystrokes[1], keystrokes[2], keystrokes[3], keystrokes[4], keystrokes[5], keystrokes[6], keystrokes[7]         
        if int(sec_fst_du) < 0:
            negUD_count += 1
        if int(sec_fst_dd) < 0:
            negUU_count += 1

        total_token_count += 1
        if fst_key_name == 'Back':
            back_count += 1
        if fst_key_name == 'LShiftKey':
            lshift_count += 1
        if fst_key_name == 'RShiftKey':
            rshift_count += 1
        if fst_key_name == 'Capital':
            capital_count += 1
        if fst_key_name == 'LControlKey':
            lctrl_count += 1
        if fst_key_name == 'RControlKey':
            rctrl_count += 1
        if fst_key_name == 'Left' or fst_key_name == 'Right':
            left_right_count += 1

    mpw = total_token_count / (((end_timestamp - start_timestamp)/1000) / 60)
    back_rate = back_count / total_token_count
    lshift_rate = lshift_count / total_token_count
    rshift_rate = rshift_count / total_token_count
    capital_rate = capital_count / total_token_count
    lctrl_rate = lctrl_count / total_token_count
    rctrl_rate = rctrl_count / total_token_count
    negUD_rate = negUD_count / total_token_count
    negUU_rate = negUU_count / total_token_count
    left_right_rate = left_right_count / total_token_count

    feature.append(mpw)
    feature.append(back_rate)
    feature.append(lshift_rate)
    feature.append(rshift_rate)
    feature.append(capital_rate)
    feature.append(lctrl_rate)
    feature.append(rctrl_rate)
    feature.append(negUD_rate)
    feature.append(negUU_rate)
    feature.append(left_right_rate)
    return feature

In [4]:
import numpy as np
import json
import torch
from torch.utils import data

class BuffaloKeyStroke(data.Dataset):
    def __init__(self, data_json, keymap_json, sequence_len=100, mean=None, std=None, u_id=1):
        with open(data_json, 'r') as f:
            data = json.load(f)
        with open(keymap_json, 'r') as f:
            self.keyname_to_id = json.load(f)
        
        if mean is None and std is None:
            data_np = []
            for squence, _ in data:
                x = list(map(lambda step: (self.keyname_to_id[step[0]], self.keyname_to_id[step[1]], step[2], step[3], step[4], step[5]), squence))
                data_np += x
            data_np = np.array(data_np)
            self.mean = np.mean(data_np, axis=0)
            self.std = np.std(data_np, axis=0)
        else:
            self.mean = mean
            self.std = std
    
        self.conv_features = []
        self.nconv_features = []
        self.labels = []
        for sequence, uid in data:
            for i, _ in enumerate(sequence[:-sequence_len]):
                self.nconv_features.append(get_non_conv_feature(sequence[i:i+sequence_len]))
                self.conv_features.append(sequence[i:i+sequence_len])
                self.labels.append(int(uid == u_id))
        self.num_positive = (np.array(self.labels) == 1).sum()
        self.num_negative = (np.array(self.labels) == 0).sum()
        self.weights = [self.num_positive if label == 0 else self.num_negative for label in self.labels]

        
    def __getitem__(self, index):
        x = self.conv_features[index]
        x = list(map(lambda step: (self.keyname_to_id[step[0]], self.keyname_to_id[step[1]], step[2], step[3], step[4], step[5]), x))
        x = (x - self.mean) / self.std
        
        z = self.nconv_features[index]
        return torch.FloatTensor(x).transpose(0, 1), torch.FloatTensor(z), self.labels[index]
    
    def __len__(self):
        return len(self.conv_features)

In [8]:
file_path = "./s01_train_s2_test/s01_baseline_train.json"
file_path1 = "./s01_train_s2_test/keyname_to_id.json"

dataset = BuffaloKeyStroke(file_path, file_path1)
x_conv, x_nconv, y = dataset[200]
print(dataset.num_positive)
print(dataset.num_negative)
# print(x_conv[0])
# print(x_nconv)

KeyboardInterrupt: 

In [5]:
import tqdm
def train(model, train_loader, criterion):
    model.train()
    epoch_loss = 0.0
    epoch_acc = 0.0
    num_batch = len(train_loader)
    for i, (batch_conv, batch_nconv, labels) in enumerate(train_loader):
        batch_conv, batch_nconv = batch_conv.cuda(), batch_nconv.cuda()
        labels = labels.cuda()
        criterion = criterion.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(batch_conv, batch_nconv)
        loss = criterion(outputs, labels.type(torch.cuda.FloatTensor))
        loss.backward()
        optimizer.step()
        # print statistics
        epoch_loss += loss.item()
        epoch_acc += outputs.detach().gt(0).long().eq(labels.detach()).float().mean()
    print('Epoch {} Train: loss= {:3f}, Acc= {:3f}'.format(epoch, epoch_loss / num_batch, epoch_acc / num_batch))
    return epoch_loss / num_batch, epoch_acc / num_batch


def test(model, test_loader, criterion):
    model.eval()
    epoch_loss = 0.0
    epoch_acc = 0.0
    num_batch = len(test_loader)
    for i, (batch_conv, batch_nconv, labels) in enumerate(test_loader):
        batch_conv, batch_nconv = batch_conv.cuda(), batch_nconv.cuda()
        labels = labels.cuda()
        criterion = criterion.cuda()

        with torch.no_grad():
            outputs = model(batch_conv, batch_nconv)
            loss = criterion(outputs, labels.type(torch.cuda.FloatTensor))
            # print statistics
            epoch_loss += loss.item()
            epoch_acc += outputs.gt(0).long().eq(labels).float().mean()
    print('Epoch {} Test: loss= {:3f}, Acc= {:3f}'.format(epoch, epoch_loss / num_batch, epoch_acc / num_batch))
    return epoch_loss / num_batch, epoch_acc / num_batch

In [7]:
from torch.utils.data import WeightedRandomSampler
from torch.utils.data import BatchSampler
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR
import os

dataset_path = ["s01_train_s2_test", "s02_train_s1_test", "s12_train_s0_test"]
for p in dataset_path: 
    file_path = "./"+p+"/baseline_train.json"
    file_path = "./"+p+"/s01_baseline_train.json"
    file_path1 = "./"+p+"/keyname_to_id.json"
    test_fp = "./baseline_test.json"
    test_fp = "./"+p+"/s01_baseline_train.json"
    kernel_optional = ["single-kernel", "multi-kernel"]
    for kernel in kernel_optional:
        
        if kernel == "single-kernel":
            ckpt_path_prefix = "./"+ p +"/ckpt_s_kernel/ckpt_{}.pth"
        else:
            ckpt_path_prefix = "./"+ p +"/ckpt_m_kernel/ckpt_{}.pth"
        
        uid_list = [i for i in range(36,76)]
        
        for uid in uid_list:
            print('UID:', uid)
            model = Net([2,3,4], 8, 10, kernel_mode=kernel).cuda()
            criterion = nn.BCEWithLogitsLoss(reduction='mean').cuda()
            optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
            lr_scheduler = StepLR(optimizer, 120)

            state_dict = {"epoch": 0, "best_model_state": model.state_dict(), "last_model_state": model.state_dict(), "best_acc": 0.0, "train_loss_epoch": [], "train_acc_epoch": [], "test_loss_epoch": [], "test_acc_epoch": []}

            # Load model
#             if os.path.exists(ckpt_path_prefix.format(uid)):
#                 state_dict = torch.load(ckpt_path_prefix.format(uid))
#                 model.load_state_dict(state_dict['last_model_state'])
#                 epoch = state_dict['epoch']
#                 print(epoch, '@#$#@@##$')
            
            train_dataset = BuffaloKeyStroke(file_path, file_path1, u_id=uid)
            train_sampler = WeightedRandomSampler(train_dataset.weights, int(train_dataset.num_positive) * 2, replacement=True)
            train_dataloader = DataLoader(train_dataset, batch_size=64, sampler=train_sampler, drop_last=True)
            
            test_dataset = BuffaloKeyStroke(test_fp, file_path1, 100, train_dataset.mean, train_dataset.std, u_id=uid)
            test_sampler = WeightedRandomSampler(test_dataset.weights, int(test_dataset.num_positive) * 2, replacement=True)
            test_dataloader = DataLoader(test_dataset, batch_size=64, sampler=test_sampler, drop_last=True)    
            
            best_test_loss = float('inf')
            best_test_acc = 0.0
            print('train_path:', p, ', kernel_mode:', kernel)
            
            for epoch in range(state_dict['epoch']+1, 70):  # loop over the dataset multiple times

                train_loss, train_acc = train(model, train_dataloader, criterion)
                test_loss, test_acc = test(model, test_dataloader, criterion)

                state_dict['train_loss_epoch'].append(train_loss)
                state_dict['train_acc_epoch'].append(train_acc)
                state_dict['test_loss_epoch'].append(test_loss)
                state_dict['test_acc_epoch'].append(test_acc)

                lr_scheduler.step()

                # Save model
                if state_dict['best_acc'] < test_acc:
                    state_dict['best_acc'] = test_acc
                    state_dict['best_model_state'] = model.state_dict()

                state_dict['last_model_state'] = model.state_dict()
                state_dict['epoch'] = epoch
                torch.save(state_dict, ckpt_path_prefix.format(uid))

            print('Finished Training')

UID: 36
train_path: s01_train_s2_test , kernel_mode: single-kernel
Epoch 1 Train: loss= 0.979071, Acc= 0.486979
Epoch 1 Test: loss= 0.703154, Acc= 0.536210
Epoch 2 Train: loss= 0.700598, Acc= 0.492684
Epoch 2 Test: loss= 0.691960, Acc= 0.539063
Epoch 3 Train: loss= 0.693798, Acc= 0.514261
Epoch 3 Test: loss= 0.684807, Acc= 0.552827
Epoch 4 Train: loss= 0.687011, Acc= 0.537946
Epoch 4 Test: loss= 0.667341, Acc= 0.598958
Epoch 5 Train: loss= 0.664561, Acc= 0.600198
Epoch 5 Test: loss= 0.573416, Acc= 0.717510
Epoch 6 Train: loss= 0.582916, Acc= 0.716022
Epoch 6 Test: loss= 0.462808, Acc= 0.800967
Epoch 7 Train: loss= 0.554066, Acc= 0.737971
Epoch 7 Test: loss= 0.420115, Acc= 0.831969
Epoch 8 Train: loss= 0.520108, Acc= 0.765501
Epoch 8 Test: loss= 0.398061, Acc= 0.845238
Epoch 9 Train: loss= 0.498212, Acc= 0.780382
Epoch 9 Test: loss= 0.363932, Acc= 0.859747
Epoch 10 Train: loss= 0.475106, Acc= 0.796751
Epoch 10 Test: loss= 0.418676, Acc= 0.820065
Epoch 11 Train: loss= 0.450018, Acc= 0.81

Epoch 20 Test: loss= 0.593369, Acc= 0.650174
Epoch 21 Train: loss= 0.517047, Acc= 0.746528
Epoch 21 Test: loss= 0.477061, Acc= 0.769097
Epoch 22 Train: loss= 0.518997, Acc= 0.743924
Epoch 22 Test: loss= 0.438811, Acc= 0.799826
Epoch 23 Train: loss= 0.496592, Acc= 0.754514
Epoch 23 Test: loss= 0.358595, Acc= 0.860069
Epoch 24 Train: loss= 0.507514, Acc= 0.756771
Epoch 24 Test: loss= 0.427282, Acc= 0.806250
Epoch 25 Train: loss= 0.495443, Acc= 0.761632
Epoch 25 Test: loss= 0.438817, Acc= 0.790799
Epoch 26 Train: loss= 0.473863, Acc= 0.777604
Epoch 26 Test: loss= 0.582081, Acc= 0.669618
Epoch 27 Train: loss= 0.467792, Acc= 0.779688
Epoch 27 Test: loss= 0.461776, Acc= 0.761979
Epoch 28 Train: loss= 0.488600, Acc= 0.774132
Epoch 28 Test: loss= 0.391894, Acc= 0.834722
Epoch 29 Train: loss= 0.469315, Acc= 0.782465
Epoch 29 Test: loss= 0.318251, Acc= 0.879861
Epoch 30 Train: loss= 0.463677, Acc= 0.785938
Epoch 30 Test: loss= 0.602568, Acc= 0.666840
Epoch 31 Train: loss= 0.461246, Acc= 0.788889

Epoch 40 Test: loss= 0.411719, Acc= 0.801153
Epoch 41 Train: loss= 0.402586, Acc= 0.828125
Epoch 41 Test: loss= 0.449877, Acc= 0.773624
Epoch 42 Train: loss= 0.386049, Acc= 0.840960
Epoch 42 Test: loss= 0.433607, Acc= 0.783668
Epoch 43 Train: loss= 0.379213, Acc= 0.840588
Epoch 43 Test: loss= 0.441097, Acc= 0.787574
Epoch 44 Train: loss= 0.373466, Acc= 0.839844
Epoch 44 Test: loss= 0.432967, Acc= 0.789249
Epoch 45 Train: loss= 0.356043, Acc= 0.859933
Epoch 45 Test: loss= 0.465760, Acc= 0.770833
Epoch 46 Train: loss= 0.380243, Acc= 0.841146
Epoch 46 Test: loss= 0.489515, Acc= 0.751488
Epoch 47 Train: loss= 0.394656, Acc= 0.837984
Epoch 47 Test: loss= 0.471824, Acc= 0.757441
Epoch 48 Train: loss= 0.379889, Acc= 0.839658
Epoch 48 Test: loss= 0.737360, Acc= 0.645089
Epoch 49 Train: loss= 0.383236, Acc= 0.834821
Epoch 49 Test: loss= 0.351373, Acc= 0.832589
Epoch 50 Train: loss= 0.399478, Acc= 0.823289
Epoch 50 Test: loss= 0.550205, Acc= 0.703683
Epoch 51 Train: loss= 0.395140, Acc= 0.830915

Epoch 60 Test: loss= 0.095147, Acc= 0.963702
Epoch 61 Train: loss= 0.179784, Acc= 0.932452
Epoch 61 Test: loss= 0.094062, Acc= 0.968269
Epoch 62 Train: loss= 0.165460, Acc= 0.943990
Epoch 62 Test: loss= 0.091381, Acc= 0.970673
Epoch 63 Train: loss= 0.179924, Acc= 0.937500
Epoch 63 Test: loss= 0.072564, Acc= 0.975962
Epoch 64 Train: loss= 0.174352, Acc= 0.939183
Epoch 64 Test: loss= 0.097010, Acc= 0.966346
Epoch 65 Train: loss= 0.173164, Acc= 0.940144
Epoch 65 Test: loss= 0.108878, Acc= 0.961058
Epoch 66 Train: loss= 0.164295, Acc= 0.942067
Epoch 66 Test: loss= 0.103231, Acc= 0.963221
Epoch 67 Train: loss= 0.161223, Acc= 0.946394
Epoch 67 Test: loss= 0.079886, Acc= 0.973558
Epoch 68 Train: loss= 0.190704, Acc= 0.928125
Epoch 68 Test: loss= 0.092500, Acc= 0.972356
Epoch 69 Train: loss= 0.161474, Acc= 0.939183
Epoch 69 Test: loss= 0.081907, Acc= 0.972115
Finished Training
UID: 40
train_path: s01_train_s2_test , kernel_mode: single-kernel
Epoch 1 Train: loss= 0.695574, Acc= 0.519737
Epoch 

Epoch 11 Train: loss= 0.532570, Acc= 0.741319
Epoch 11 Test: loss= 0.457656, Acc= 0.806818
Epoch 12 Train: loss= 0.541368, Acc= 0.740057
Epoch 12 Test: loss= 0.448090, Acc= 0.815814
Epoch 13 Train: loss= 0.523822, Acc= 0.752525
Epoch 13 Test: loss= 0.641464, Acc= 0.637784
Epoch 14 Train: loss= 0.537542, Acc= 0.743371
Epoch 14 Test: loss= 0.449732, Acc= 0.816288
Epoch 15 Train: loss= 0.517606, Acc= 0.759312
Epoch 15 Test: loss= 0.455305, Acc= 0.794823
Epoch 16 Train: loss= 0.503721, Acc= 0.770675
Epoch 16 Test: loss= 0.418407, Acc= 0.826389
Epoch 17 Train: loss= 0.524025, Acc= 0.753630
Epoch 17 Test: loss= 0.455385, Acc= 0.793087
Epoch 18 Train: loss= 0.514960, Acc= 0.758996
Epoch 18 Test: loss= 0.449035, Acc= 0.799558
Epoch 19 Train: loss= 0.518571, Acc= 0.747790
Epoch 19 Test: loss= 0.657076, Acc= 0.618371
Epoch 20 Train: loss= 0.499770, Acc= 0.766572
Epoch 20 Test: loss= 0.411566, Acc= 0.831597
Epoch 21 Train: loss= 0.506802, Acc= 0.765941
Epoch 21 Test: loss= 0.421367, Acc= 0.834122

...

Epoch 22 Test: loss= 0.411119, Acc= 0.816947
Epoch 23 Train: loss= 0.443288, Acc= 0.796154
Epoch 23 Test: loss= 0.350210, Acc= 0.865745
Epoch 24 Train: loss= 0.435014, Acc= 0.800481
Epoch 24 Test: loss= 0.346937, Acc= 0.879447
Epoch 25 Train: loss= 0.430004, Acc= 0.810697
Epoch 25 Test: loss= 1.093647, Acc= 0.498678
Epoch 26 Train: loss= 0.434492, Acc= 0.805168
Epoch 26 Test: loss= 0.368248, Acc= 0.848558
Epoch 27 Train: loss= 0.449949, Acc= 0.789303
Epoch 27 Test: loss= 0.309150, Acc= 0.881490
Epoch 28 Train: loss= 0.424508, Acc= 0.811058
Epoch 28 Test: loss= 0.358895, Acc= 0.860817
Epoch 29 Train: loss= 0.425051, Acc= 0.810337
Epoch 29 Test: loss= 0.352075, Acc= 0.838221
Epoch 30 Train: loss= 0.430848, Acc= 0.806370
Epoch 30 Test: loss= 0.301977, Acc= 0.902644
Epoch 31 Train: loss= 0.393548, Acc= 0.828365
Epoch 31 Test: loss= 0.329206, Acc= 0.875120
Epoch 32 Train: loss= 0.411874, Acc= 0.819832
Epoch 32 Test: loss= 0.368436, Acc= 0.830529
Epoch 33 Train: loss= 0.416915, Acc= 0.811899

Epoch 42 Test: loss= 0.253956, Acc= 0.908467
Epoch 43 Train: loss= 0.381212, Acc= 0.838260
Epoch 43 Test: loss= 0.238829, Acc= 0.911951
Epoch 44 Train: loss= 0.389538, Acc= 0.828864
Epoch 44 Test: loss= 0.229998, Acc= 0.919447
Epoch 45 Train: loss= 0.383026, Acc= 0.837943
Epoch 45 Test: loss= 0.276603, Acc= 0.894954
Epoch 46 Train: loss= 0.378772, Acc= 0.842378
Epoch 46 Test: loss= 0.274082, Acc= 0.895270
Epoch 47 Train: loss= 0.367677, Acc= 0.843328
Epoch 47 Test: loss= 0.227972, Acc= 0.917441
Epoch 48 Train: loss= 0.353465, Acc= 0.853146
Epoch 48 Test: loss= 0.253974, Acc= 0.896009
Epoch 49 Train: loss= 0.382326, Acc= 0.840160
Epoch 49 Test: loss= 0.230303, Acc= 0.910579
Epoch 50 Train: loss= 0.357831, Acc= 0.849345
Epoch 50 Test: loss= 0.249625, Acc= 0.896537
Epoch 51 Train: loss= 0.357941, Acc= 0.848501
Epoch 51 Test: loss= 0.272219, Acc= 0.886508
Epoch 52 Train: loss= 0.342889, Acc= 0.859481
Epoch 52 Test: loss= 0.220280, Acc= 0.926415
Epoch 53 Train: loss= 0.358730, Acc= 0.850824

Epoch 62 Test: loss= 0.189517, Acc= 0.922905
Epoch 63 Train: loss= 0.375315, Acc= 0.836195
Epoch 63 Test: loss= 0.233593, Acc= 0.904190
Epoch 64 Train: loss= 0.363203, Acc= 0.849588
Epoch 64 Test: loss= 0.202304, Acc= 0.922218
Epoch 65 Train: loss= 0.349783, Acc= 0.849588
Epoch 65 Test: loss= 0.214410, Acc= 0.915007
Epoch 66 Train: loss= 0.320172, Acc= 0.872081
Epoch 66 Test: loss= 0.188878, Acc= 0.927713
Epoch 67 Train: loss= 0.322180, Acc= 0.864698
Epoch 67 Test: loss= 0.195654, Acc= 0.923420
Epoch 68 Train: loss= 0.333807, Acc= 0.863153
Epoch 68 Test: loss= 0.225379, Acc= 0.902129
Epoch 69 Train: loss= 0.323819, Acc= 0.864698
Epoch 69 Test: loss= 0.223203, Acc= 0.902301
Finished Training
UID: 36
train_path: s01_train_s2_test , kernel_mode: multi-kernel
Epoch 1 Train: loss= 1.900685, Acc= 0.484499
Epoch 1 Test: loss= 0.743531, Acc= 0.452753
Epoch 2 Train: loss= 0.760902, Acc= 0.476191
Epoch 2 Test: loss= 0.704749, Acc= 0.483383
Epoch 3 Train: loss= 0.705126, Acc= 0.506572
Epoch 3 Tes

Epoch 13 Train: loss= 0.525466, Acc= 0.741493
Epoch 13 Test: loss= 0.411069, Acc= 0.848264
Epoch 14 Train: loss= 0.489007, Acc= 0.772743
Epoch 14 Test: loss= 0.396878, Acc= 0.848611
Epoch 15 Train: loss= 0.480468, Acc= 0.787326
Epoch 15 Test: loss= 0.417657, Acc= 0.817708
Epoch 16 Train: loss= 0.456052, Acc= 0.800347
Epoch 16 Test: loss= 0.802894, Acc= 0.611458
Epoch 17 Train: loss= 0.433023, Acc= 0.811979
Epoch 17 Test: loss= 0.692898, Acc= 0.655556
Epoch 18 Train: loss= 0.429558, Acc= 0.817361
Epoch 18 Test: loss= 0.872189, Acc= 0.596701
Epoch 19 Train: loss= 0.422393, Acc= 0.818924
Epoch 19 Test: loss= 0.536990, Acc= 0.747569
Epoch 20 Train: loss= 0.409102, Acc= 0.827951
Epoch 20 Test: loss= 0.829794, Acc= 0.622743
Epoch 21 Train: loss= 0.385227, Acc= 0.841493
Epoch 21 Test: loss= 0.499148, Acc= 0.753646
Epoch 22 Train: loss= 0.367733, Acc= 0.850694
Epoch 22 Test: loss= 0.446487, Acc= 0.782639
Epoch 23 Train: loss= 0.381046, Acc= 0.840625
Epoch 23 Test: loss= 0.885614, Acc= 0.607813

RuntimeError: CUDA error: unspecified launch failure

In [25]:
uid_list = [i for i in range(1,36)]
acc_avg, acc_sum = 0, 0
for i in uid_list:
    state_dict = torch.load('./s01_train_s2_test/ckpt_s_kernel/ckpt_{}.pth'.format(i))
    print("Uid:", i, "Best_acc", float(state_dict['best_acc']))
    acc_sum += float(state_dict['best_acc'])
print('acc avg:', acc_sum/35)

Uid: 1 Best_acc 0.5985440611839294
Uid: 2 Best_acc 0.7599431872367859
Uid: 3 Best_acc 0.8614130616188049
Uid: 4 Best_acc 0.5666667222976685
Uid: 5 Best_acc 0.9164062738418579
Uid: 6 Best_acc 0.9359681606292725
Uid: 7 Best_acc 0.8417552709579468
Uid: 8 Best_acc 0.7009804248809814
Uid: 9 Best_acc 0.8416510820388794
Uid: 10 Best_acc 0.4964887797832489
Uid: 11 Best_acc 0.9076923131942749
Uid: 12 Best_acc 0.8274038434028625
Uid: 13 Best_acc 0.7776268124580383
Uid: 14 Best_acc 0.975900411605835
Uid: 15 Best_acc 0.9183728694915771
Uid: 16 Best_acc 0.5705819129943848
Uid: 17 Best_acc 0.7042545080184937
Uid: 18 Best_acc 0.9591346383094788
Uid: 19 Best_acc 0.970552921295166
Uid: 20 Best_acc 0.9055989980697632
Uid: 21 Best_acc 0.9580792188644409
Uid: 22 Best_acc 0.9597039818763733
Uid: 23 Best_acc 0.880126953125
Uid: 24 Best_acc 0.7212010025978088
Uid: 25 Best_acc 0.9002840518951416
Uid: 26 Best_acc 0.6306818127632141
Uid: 27 Best_acc 0.875744104385376
Uid: 28 Best_acc 0.7762277126312256
Uid: 29 

In [29]:
uid_list = [i for i in range(1,36)]
acc_avg, acc_sum = 0, 0
for i in uid_list:
    state_dict = torch.load('./s01_train_s2_test/ckpt_m_kernel/ckpt_{}.pth'.format(i))
    print("Uid:", i, "Best_acc", state_dict['best_acc'])
    acc_sum += float(state_dict['best_acc'])    
print('acc avg:', acc_sum/35)

Uid: 1 Best_acc tensor(0.6618, device='cuda:0')
Uid: 2 Best_acc tensor(0.7028, device='cuda:0')
Uid: 3 Best_acc tensor(0.9423, device='cuda:0')
Uid: 4 Best_acc tensor(0.6169, device='cuda:0')
Uid: 5 Best_acc tensor(0.9333, device='cuda:0')
Uid: 6 Best_acc tensor(0.9400, device='cuda:0')
Uid: 7 Best_acc tensor(0.8916, device='cuda:0')
Uid: 8 Best_acc tensor(0.7273, device='cuda:0')
Uid: 9 Best_acc tensor(0.7882, device='cuda:0')
Uid: 10 Best_acc tensor(0.5730, device='cuda:0')
Uid: 11 Best_acc tensor(0.9502, device='cuda:0')
Uid: 12 Best_acc tensor(0.8793, device='cuda:0')
Uid: 13 Best_acc tensor(0.7298, device='cuda:0')
Uid: 14 Best_acc tensor(0.9947, device='cuda:0')
Uid: 15 Best_acc tensor(0.9135, device='cuda:0')
Uid: 16 Best_acc tensor(0.5374, device='cuda:0')
Uid: 17 Best_acc tensor(0.7026, device='cuda:0')
Uid: 18 Best_acc tensor(0.9498, device='cuda:0')
Uid: 19 Best_acc tensor(0.9246, device='cuda:0')
Uid: 20 Best_acc tensor(0.9131, device='cuda:0')
Uid: 21 Best_acc tensor(0.997

In [45]:
x = torch.randn(2, 3)
print(x)
print('================')
a = torch.cat((x, x, x), 0)
print(a , a.shape)
print('================')
b = torch.cat((x, x, x), -1)
print(b, b.shape)

y = torch.randn(6, 2, 8)
z = torch.randn(6, 2, 12)
t = torch.randn(6, 2, 12)
yz = torch.cat((y, z, t), -1)
print(yz.shape)


tensor([[ 0.8913, -1.1462, -0.4117],
        [ 0.8636, -0.2382, -0.5940]])
tensor([[ 0.8913, -1.1462, -0.4117],
        [ 0.8636, -0.2382, -0.5940],
        [ 0.8913, -1.1462, -0.4117],
        [ 0.8636, -0.2382, -0.5940],
        [ 0.8913, -1.1462, -0.4117],
        [ 0.8636, -0.2382, -0.5940]]) torch.Size([6, 3])
tensor([[ 0.8913, -1.1462, -0.4117,  0.8913, -1.1462, -0.4117,  0.8913, -1.1462,
         -0.4117],
        [ 0.8636, -0.2382, -0.5940,  0.8636, -0.2382, -0.5940,  0.8636, -0.2382,
         -0.5940]]) torch.Size([2, 9])
torch.Size([6, 2, 32])
